# 가장 빠른 길 찾기

## 최단 경로 알고리즘
* 상황에 따라 제일 효율적인 알고리즘이 다름
    * 한 지점에서 다른 특정 지점? 모든 지점에서 모든 지점?
    * 다익스트라, 플로이드-워셜 등

## 다익스트라 최단 경로 알고리즘
* 특정 노드에서 다른 노드로 가는 최단 경로를 구해 줌
* 음의 간선이 없을 때 정상적으로 동작
* 기본적으로 그리디 알고리즘 - 매번 가장 비용이 적은 노드 선택


**동작 원리**
1. 출발 노드를 설정
2. 최단 거리 테이블을 초기화
3. 방문하지 않은 노드 중, 최단 거리가 가장 짧은 노드를 선택
4. 해당 노드를 거쳐 다른 노드로 가는 비용을 계산하여 최단 거리 테이블 갱신
5. 3, 4단계를 반복
* 최단 거리 테이블: 각 노드에 대한 현재까지의 최단 거리를 저장하는 1차원 리스트, 계속 갱신


* 3단계에서 선택된 노드는 최단 거리가 완전히 선택된 노드, 더 이상 알고리즘을 반복해도 최단 거리가 줄어들지 않음
* **3-4단계를 1번 반복할 때마다, 하나의 노드에 대한 최단 거리를 확실히 찾음**

**방법 1: $O(V^2)$**
* $V$는 노드의 개수
* 3단계를 반복할 때마다 최단 거리 테이블을 순차 탐색
* 노드가 10,000개를 넘어가면 문제 해결 어려움

In [2]:
import sys
# input = lambda: sys.stdin.readline().rstrip()
INF = int(1e9) # 정수 처리를 해 주는 게 안전

n, m = map(int, input().split()) # 노드, 간선의 개수
start = int(input()) # 시작노드 번호
graph = [[] for _ in range(n + 1)] # 그래프
visited = [False] * (n + 1) # 방문 여부
distance = [INF] * (n + 1) # 최단거리 테이블

for _ in range(m):
    # a->b 비용: c
    a, b, c = map(int, input().split())
    graph[a].append((b, c))

# 3단계: 방문하지 않은 노드 중 가장 최단거리가 짧은 노드
def get_smallest_node():
    min_value = INF
    smallest = 0
    # O(V)번에 걸쳐 최단 거리가 짧은 노드를 선형 탐색
    for i in range(1, n + 1):
        if distance[i] < min_value and not visited[i]:
            min_value = distance[i]
            smallest = i
    return smallest

def djikstra(start):
    # 1/2단계: 출발노드 설정, 초기화
    distance[start] = 0
    visited[start] = True

    # 4단계: 해당 노드를 거쳐 다른 노드로 가는 비용을 계산해 최단 거리를 계산
    for next in graph[start]:
        distance[next[0]] = next[1]

    # 시작 노드를 제외한 노드에 대해 3, 4단계 반복
    # O(V)번에 걸쳐 현재 노드와 연결된 노드 확인
    for _ in range(n - 1):
        curr = get_smallest_node()
        visited[curr] = True

        for next in graph[curr]:
            cost = distance[curr] + next[1]
            if cost < distance[next[0]]:
                distance[next[0]] = cost

djikstra(start)

for i in range(1, n + 1):
    if distance[i] == INF:
        print("무한")
    else:
        print(distance[i])

6 11
1
1 2 2
1 3 5
1 4 1
2 3 3
2 4 2
3 2 3
3 6 5
4 3 3
4 5 1
5 3 1
5 6 2
0
2
3
1
2
4


In [7]:
# 복습, 직접 한 번 구현해 봅시다

INF = int(1e9)
n, m = map(int, input().split()) # 노드, 간선 수
start = int(input()) # 어디서시작?
graph = [[] for _ in range(n + 1)] # 그래프
visited = [False] * (n + 1) # 방문여부
distance = [INF] * (n + 1) # 최단거리 테이블

for _ in range(m):
    s, e, c = map(int, input().split())
    graph[s].append((e, c))

# 최단거리 + 방문안함
def next_dest():
    shortest = INF
    s_index = 0
    for i in range(1, n + 1):
        if not visited[i] and distance[i] < shortest:
            shortest = distance[i]
            s_index = i
    return s_index

# 다익스트라
def djikstra(start):
    # 초기화
    visited[start] = True
    distance[start] = 0

    # 거리 갱신
    for next, next_cost in graph[start]:
        distance[next] = next_cost

    # 이걸 나머지 애들한테도 반복
    for _ in range(n - 1):
        curr = next_dest()
        visited[curr] = True
        for next, next_cost in graph[curr]:
            pass_cost = distance[curr] + next_cost
            distance[next] = min(pass_cost, distance[next])

djikstra(start)

for i in range(1, n + 1):
    if distance[i] == INF:
        print("무한")
    else:
        print(distance[i])

6 11
1
1 2 2
1 3 5
1 4 1
2 3 3
2 4 2
3 2 3
3 6 5
4 3 3
4 5 1
5 3 1
5 6 2
0
2
3
1
2
4


## 우선순위 큐 (힙)

**방법 2: O(E log V)**
* $E$는 간선의 개수, $V$는 노드의 개수
* 최단 거리가 가장 짧은 노드를 빠르게 찾기 -> 우선순위 큐 사용
* 3단계 - 기존 순차 탐색에선 $O(V)$ 필요, 우선순위 큐 사용 시 $O(log V)$

**힙**
* 우선순위 큐를 구현하기 위해 사용하는 자료구조
* 우선순위가 가장 높은 데이터를 가장 먼저 삭제
* `heapq` 패키지 사용
* `(가치, 물건)`으로 묶어 우선순위 큐에 넣기 -> 1번째 원소를 기준으로 우선순위 설정, 항상 가치가 높은 물건이 먼저 나오게 됨


* 최소 힙: 값이 낮은 데이터 먼저 삭제 (Python 기본, 다익스트라에서는 비용이 적은 노드 방문을 위해 사용)
* 최대 힙: 값이 큰 데이터 먼저 (C++ 기본, Python에선 우선순위 해당값에 음수 부호를 붙인 후, 꺼낸 후 돌리는 방식을 사용)
* 힙 사용 시, 데이터 넣고 뺄 때 $O(log N)$
* $N$개의 데이터를 모두 넣고 모두 꺼낼 때 $O(N log N)$

**우선순위 큐를 이용한 다익스트라 알고리즘**
* 3단계의 현재 가장 가까운 노드를 저장하기 위한 목적으로, 우선순위 큐를 추가로 이용
* 우선순위 큐에서 노드를 꺼낸 뒤 이미 처리한 적이 있다면 무시, 아직 처리하지 않은 노드에 대해서만 처리

In [9]:
import sys
import heapq
# input = lambda: sys.stdin.readline().rstrip()
INF = int(1e9) # 정수 처리를 해 주는 게 안전

n, m = map(int, input().split()) # 노드, 간선의 개수
start = int(input()) # 시작노드 번호
graph = [[] for _ in range(n + 1)] # 그래프
distance = [INF] * (n + 1) # 최단거리 테이블

for _ in range(m):
    # a->b 비용: c
    a, b, c = map(int, input().split())
    graph[a].append((b, c))


def djikstra(start):
    q = []

    # 1/2단계: 출발노드 설정, 초기화
    heapq.heappush(q, (0, start)) # 거리, 노드번호
    distance[start] = 0

    # 3단계: 가장 최단거리가 짧은 노드 확인, 이미 처리된 노드라는 뜻
    while q:
        dist, curr = heapq.heappop(q)
        if distance[curr] < dist:
            continue # 이미 처리된 노드라는 뜻

        # 4단계: 해당 노드를 거쳐 다른 노드로 가는 비용을 계산해 최단 거리를 계산
        for next in graph[curr]:
            cost = distance[curr] + next[1]
            if cost < distance[next[0]]:
                distance[next[0]] = cost
                heapq.heappush(q, (cost, next[0]))

djikstra(start)

for i in range(1, n + 1):
    if distance[i] == INF:
        print("무한")
    else:
        print(distance[i])

6 11
1
1 2 2
1 3 5
1 4 1
2 3 3
2 4 2
3 2 3
3 6 5
4 3 3
4 5 1
5 3 1
5 6 2
0
2
3
1
2
4


* 힙에 최대 $E$개의 원소를 우선순위 큐에 넣었다 뺌 -> $O(E log E)$
* $E \leq V^2$이므로, $O(E log V^2) = O(2E log V) = O(E log V)$로도 볼 수 있음

## 플로이드 워셜 알고리즘
* 모든 지점에서 다른 모든 지점까지의 최단경로를 모두 구해야 하는 경우
* 노드의 개수가 $N$개일 때, $N$단계 수행. 단계마다 $O(N^2)$의 연산을 통해 현재 노드를 거쳐 가는 모든 경로를 고려 -> $O(N^3)$
* 2차원 리스트에 최단 거리 정보 저장 -> 모든 노드에 대해 다른 모든 노드로 가는 최단 정보를 담아야 함


* 다이나믹 프로그래밍: $N$단계 동안 점화식에 맞게 2차원 리스트 갱신
* 각 단계에선 해당 노드를 거쳐 가는 경우 고려 (e.g., A -> 1번 노드 -> B)
* 현재 확인 중인 노드 제외, $N-1$갠 노드 중에서 서로 다른 노드 $(A, B)$ 쌍 선택 후 확인. $O(P(N-1, 2)) = O(N^2)$
* 각 단계 $k$에 대해, $D_{ab} = min(D_{ab}, D_{ak} + D_{kb})$의 점화식으로 갱신

In [10]:
INF = int(1e9)
n = int(input()) # 노드 수
m = int(input()) # 간선 수
graph = [[INF] * (n + 1) for _ in range(n + 1)]

# 자기 자신으로 가는 노드 -> 0
for a in range(1, n + 1):
    for b in range(1, n + 1):
        if a == b:
            graph[a][b] = 0

# 각 간선의 정보
for _ in range(m):
    a, b, c = map(int, input().split())
    graph[a][b] = c

for k in range(1, n + 1):
    for a in range(1, n + 1):
        for b in range(1, n + 1):
            # 위의 점화식
            graph[a][b] = min(graph[a][b], graph[a][k] + graph[k][b])

# 결과 출력
for a in range(1, n + 1):
    for b in range(1, n + 1):
        if graph[a][b] == INF:
            print("무한", end=" ")
        else:
            print(graph[a][b], end=" ")
    print()

4
7
1 2 4
1 4 6
2 1 3
2 3 7
3 1 5
3 4 4
4 3 2
0 4 8 6 
3 0 7 9 
5 9 0 4 
7 11 2 0 


# [실전] 미래 도시

In [24]:
import heapq

INF = int(1e9)
n, m = map(int, input().split())
graph = [[] for _ in range(n + 1)]

for _ in range(m):
    a, b = map(int, input().split())
    graph[a].append((b, 1))
    graph[b].append((a, 1))

x, k = map(int, input().split())

def djikstra(start, end):
    q = []
    distance = [INF for _ in range(n + 1)]
    heapq.heappush(q, (0, start))
    distance[start] = 0

    while q:
        curr_dist, curr = heapq.heappop(q)

        if distance[curr] < curr_dist:
            continue

        for next, next_dist in graph[curr]:
            cost = curr_dist + next_dist
            if cost < distance[next]:
                distance[next] = cost
                heapq.heappush(q, (distance[next], next))
    return distance[end]

dist_a = djikstra(1, k)
dist_b = djikstra(k, x)

if dist_a == INF or dist_b == INF:
    print(-1)
else:
    print(dist_a + dist_b)

5 7
1 2
1 3
1 4
2 4
3 4
3 5
4 5
4 5
3


* 지금 플로이드 워셜 문제를 넌 다익스트라로 푼 것이냐
* 뭐... 헷갈렸을 수 있지. 근데 $N = 100$으로 넉넉한 편이였으니 플로이드가 나았을 것임.

In [29]:
import heapq

INF = int(1e9)
n, m = map(int, input().split())
graph = [[INF] * (n + 1) for _ in range(n + 1)]

for a in range(1, n + 1):
    for b in range(1, n + 1):
        if a == b:
            graph[a][b] = 0

for _ in range(m):
    a, b = map(int, input().split())
    graph[a][b] = 1
    graph[b][a] = 1

x, k = map(int, input().split())

for c in range(1, n + 1):
    for a in range(1, n + 1):
        for b in range(1, n + 1):
            graph[a][b] = min(graph[a][b], graph[a][c] + graph[c][b])

dist = graph[1][k] + graph[k][x]

if dist >= INF:
    print(-1)

else:
    print(dist)

5 7
1 2
1 3
1 4
2 4
3 4
3 5
4 5
4 5
3


# [실전] 전보

In [39]:
import heapq

INF = int(1e9)
n, m, c = map(int, input().split())
graph = [[] for _ in range(n + 1)]
distance = [INF] * (n + 1)

for _ in range(m):
    x, y, z = map(int, input().split())
    graph[x].append((z, y)) # 거리, 목적지

queue = []
heapq.heappush(queue, (0, c)) # 거리, 노드 번호
distance[c] = 0

while queue:
    c_dist, curr = heapq.heappop(queue)
    if distance[curr] < c_dist:
        continue
    for n_dist, next in graph[curr]:
        cost = c_dist + n_dist

        if cost < distance[next]:
            distance[next] = cost
            heapq.heappush(queue, (cost, next))

count = 0
time = 0

for i in range(1, n+1):
    if distance[i] >= INF or i == c:
        continue
    else:
        count += 1
        if distance[i] > time:
            time = distance[i]

print(count, time)

3 2 1
1 2 4
1 3 2
2 4


* 전형적 다익스트라 문제다.
* $N$, $M$의 범위가 크다? 우선순위 큐다.